## Simple RAG Application

### Document Loader

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
import bs4
urls = ["https://transformer-circuits.pub/2025/attribution-graphs/biology.html","https://transformer-circuits.pub/2025/attribution-graphs/biology.html#introduction"]
loader = WebBaseLoader(urls)
text_document = loader.load()
text_document[0]

### Loading API Keys

In [3]:
from langchain_ollama import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
import os


### Splitting the Document Text

In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200,chunk_overlap=50)
splitted_document = text_splitter.split_documents(text_document)
# splitted_document[:5]

### Embedding the Document in FAISS DB

In [6]:
from langchain_community.vectorstores import Chroma
embedder = OllamaEmbeddings(model="all-minilm")
db = Chroma.from_documents(
    splitted_document,
    embedder,
    persist_directory="/rag_db")

### Querying Vector Database

In [ ]:
query = input("Ask the question within the context of information sources:")
result = db.similarity_search(query)
# result[1].page_content
result

In [6]:
retriever = db.as_retriever(search_kwargs={"k": 3})

In [7]:
from transformers import pipeline
from langchain.prompts import PromptTemplate
from langchain_ollama import ChatOllama
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
llm = ChatOllama(
    model="gemma3:1b",
    temperature=0.3
)

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history",return_messages=True)
rag_chain = ConversationalRetrievalChain.from_llm(
    llm = llm,
    retriever = retriever,
    memory = memory
)
while True:
    query = input()
    if query.lower == "quit":
        break
    else:
        response = rag_chain.invoke(query)
        print(response['answer'])